In [2]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [4]:
z= torch.FloatTensor([1,2,3])
y= F.softmax(z,dim=0)
print(y)
print(y.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [10]:
z=torch.rand(3,5, requires_grad=True)
h=F.softmax(z,dim=1)
print(h)
print(h.sum(dim=1))

tensor([[0.2939, 0.1425, 0.1737, 0.1868, 0.2029],
        [0.1407, 0.1442, 0.2699, 0.1828, 0.2625],
        [0.1773, 0.2030, 0.1923, 0.2066, 0.2207]], grad_fn=<SoftmaxBackward0>)
tensor([1., 1., 1.], grad_fn=<SumBackward1>)


In [14]:
y=torch.randint(5,(3,)).long()
print(y)
y_one_hot=torch.zeros_like(h)
print(y_one_hot)
y_one_hot.scatter_(1,y.unsqueeze(1),1)
print(y_one_hot)

tensor([1, 1, 4])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])


In [16]:
print(y.unsqueeze(1))

tensor([[1],
        [1],
        [4]])


In [17]:
cost=(y_one_hot * -torch.log(h)).sum(dim=1).mean()
print(cost)

tensor(1.7985, grad_fn=<MeanBackward0>)


In [18]:
torch.log(F.softmax(z,dim=1))

tensor([[-1.2244, -1.9481, -1.7502, -1.6775, -1.5948],
        [-1.9613, -1.9367, -1.3099, -1.6991, -1.3377],
        [-1.7301, -1.5944, -1.6486, -1.5768, -1.5107]], grad_fn=<LogBackward0>)

In [19]:
F.log_softmax(z,dim=1)

tensor([[-1.2244, -1.9481, -1.7502, -1.6775, -1.5948],
        [-1.9613, -1.9367, -1.3099, -1.6991, -1.3377],
        [-1.7301, -1.5944, -1.6486, -1.5768, -1.5107]],
       grad_fn=<LogSoftmaxBackward0>)

In [25]:
(y_one_hot * -torch.log(F.softmax(z,dim=1))).sum(dim=1).mean()

tensor(1.7985, grad_fn=<MeanBackward0>)

In [24]:
(y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.7985, grad_fn=<MeanBackward0>)

In [26]:
F.nll_loss(F.log_softmax(z, dim=1),y)

tensor(1.7985, grad_fn=<NllLossBackward0>)

In [27]:
F.cross_entropy(z,y)

tensor(1.7985, grad_fn=<NllLossBackward0>)

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [28]:
import numpy as np
import matplotlib.pyplot as plt


In [38]:
torch.manual_seed(1)
x_train= [[1,2,1,1],
          [2,1,3,2],
          [3,1,3,4],
          [4,1,5,5],
          [1,7,5,5],
          [1,2,5,6],
          [1,6,6,6],
          [1,7,7,7]
          ]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [50]:
y_one_hot=torch.zeros(8,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
print(y, y_one_hot)

tensor([1, 1, 4]) tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [51]:
# low_level 구현
W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros((1,3),requires_grad=True)
print(W,b)
optimizer=optim.SGD([W,b], lr=0.1)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True) tensor([[0., 0., 0.]], requires_grad=True)


In [55]:
n_epochs=1000
for epoch in range(n_epochs):
    y_hat=F.softmax(x_train.matmul(W)+b, dim=1)
    cost=(y_one_hot * -torch.log(y_hat)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("epoch=",epoch, "cost=",cost.item())

epoch= 0 cost= 0.24815471470355988
epoch= 100 cost= 0.24815471470355988
epoch= 200 cost= 0.24815471470355988
epoch= 300 cost= 0.24815471470355988
epoch= 400 cost= 0.24815471470355988
epoch= 500 cost= 0.24815471470355988
epoch= 600 cost= 0.24815471470355988
epoch= 700 cost= 0.24815471470355988
epoch= 800 cost= 0.24815471470355988
epoch= 900 cost= 0.24815471470355988


In [72]:
# high_level 구현

W=torch.zeros((4,3),requires_grad=True)
b=torch.zeros((1,3),requires_grad=True)
print(W,b)
optimizer=optim.SGD([W,b], lr=0.1)

for epoch in range(n_epochs):
    z=x_train.matmul(W) + b
    cost=F.cross_entropy(z,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("epoch=",epoch, "cost=",cost.item())


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], requires_grad=True) tensor([[0., 0., 0.]], requires_grad=True)
epoch= 0 cost= 1.0986123085021973
epoch= 100 cost= 0.7041995525360107
epoch= 200 cost= 0.6229995489120483
epoch= 300 cost= 0.5657169818878174
epoch= 400 cost= 0.5152913928031921
epoch= 500 cost= 0.46766164898872375
epoch= 600 cost= 0.4212779998779297
epoch= 700 cost= 0.3754015564918518
epoch= 800 cost= 0.3297656178474426
epoch= 900 cost= 0.2850722670555115


In [76]:
model=nn.Linear(4,3)
optimizer=optim.SGD(model.parameters(),lr=0.01)

for epoch in range(n_epochs):
    y_hat=model(x_train)
    cost=F.cross_entropy(y_hat,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("epoch=",epoch, "cost=",cost.item())

epoch= 0 cost= 2.1878204345703125
epoch= 100 cost= 0.7547277808189392
epoch= 200 cost= 0.6898152232170105
epoch= 300 cost= 0.6515492796897888
epoch= 400 cost= 0.6231000423431396
epoch= 500 cost= 0.6001359820365906
epoch= 600 cost= 0.5808707475662231
epoch= 700 cost= 0.5643244385719299
epoch= 800 cost= 0.5498682856559753
epoch= 900 cost= 0.5370631217956543


In [77]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)

    def forward(self,x):
        return self.linear(x)

In [78]:
model=SoftmaxClassifierModel()
optimizer=optim.SGD(model.parameters(), lr=0.01)

for epoch in range(n_epochs):
    y_hat=model(x_train)
    cost=F.cross_entropy(y_hat,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("epoch=",epoch, "cost=",cost.item())

epoch= 0 cost= 1.7170772552490234
epoch= 100 cost= 0.8775950074195862
epoch= 200 cost= 0.757685661315918
epoch= 300 cost= 0.6942103505134583
epoch= 400 cost= 0.6526139974594116
epoch= 500 cost= 0.6221368908882141
epoch= 600 cost= 0.598246157169342
epoch= 700 cost= 0.5786740183830261
epoch= 800 cost= 0.5621402859687805
epoch= 900 cost= 0.5478534698486328
